In [32]:
## compute entropy of spectrum signals as biomarkers
## udpates on Apr 11 2022:
## save spectrum as time points by frequency for each subject.
## calculate spectrum only on test set.

import glob
import pandas as pd
import numpy as np
import re
import os

## read harmonics and time signals:
## ocd data:
input_harmonics_wavelets = 'ocd_out02_harmonics'
input_time_signal = 'ocd_out01_correlation_matrix_ocd'
output_dir = 'ocd_out03_spectrum_power_ocd'

## Notes:
# got NaN values in wavelets (e.g for ROI 3 all values are nan), 
# but time signals does not have nans...
# need to check where those NaNs are derived...

if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

harmonic_wavelets_files = glob.glob(input_harmonics_wavelets + '/CommonHarWavelets_Harmonics*.csv')
harmonic_wavelets_files.sort()

## OCD data:
time_signal_files = glob.glob(input_time_signal + '/ROISignals*.csv')
time_signal_files.sort()

## HCP data:
# time_signal_files = glob.glob(input_time_signal + '/TimeSeries*.csv')


In [33]:
time_signal_files

['ocd_out01_correlation_matrix_ocd/ROISignals_sub_001.csv',
 'ocd_out01_correlation_matrix_ocd/ROISignals_sub_003.csv',
 'ocd_out01_correlation_matrix_ocd/ROISignals_sub_004.csv',
 'ocd_out01_correlation_matrix_ocd/ROISignals_sub_007.csv',
 'ocd_out01_correlation_matrix_ocd/ROISignals_sub_012.csv',
 'ocd_out01_correlation_matrix_ocd/ROISignals_sub_013.csv',
 'ocd_out01_correlation_matrix_ocd/ROISignals_sub_016.csv',
 'ocd_out01_correlation_matrix_ocd/ROISignals_sub_018.csv',
 'ocd_out01_correlation_matrix_ocd/ROISignals_sub_019.csv',
 'ocd_out01_correlation_matrix_ocd/ROISignals_sub_020.csv',
 'ocd_out01_correlation_matrix_ocd/ROISignals_sub_022.csv',
 'ocd_out01_correlation_matrix_ocd/ROISignals_sub_023.csv',
 'ocd_out01_correlation_matrix_ocd/ROISignals_sub_024.csv',
 'ocd_out01_correlation_matrix_ocd/ROISignals_sub_027.csv',
 'ocd_out01_correlation_matrix_ocd/ROISignals_sub_029.csv',
 'ocd_out01_correlation_matrix_ocd/ROISignals_sub_031.csv',
 'ocd_out01_correlation_matrix_ocd/ROISi

In [34]:
# for each wavelets (corresponding to a brain region) calcluate inner product with roi signals for each subject:

spectrum_list = [] # nested array with row: brain region and column: subject.
brain_region_list = []
subject_id_list = []

fill_subject_id = True 

for file_wavelets in harmonic_wavelets_files:
    print(file_wavelets)
    spectrum_sub_list = []
    wavelets = pd.read_csv(file_wavelets, header = None)
    
    if wavelets.isna().any(axis = None):
        print('Nan values for wavelets:' + file_wavelets)
        spectrum_list.append(spectrum_sub_list)
        continue
    
    brain_region = file_wavelets[-7:-4]
    brain_region_list.append(brain_region)
    
    for file_time_signal in time_signal_files:
        
        ## adni:
        subject_id = re.search('ROISignals_(.*).csv', file_time_signal).group(1)
        time_signal = pd.read_csv(file_time_signal, header = None, sep = ',')
        # time_signal.drop(columns = ['Unnamed: 264'], inplace = True)
        time_signal.columns = time_signal.columns.astype(int)
        time_signal = time_signal.sort_index(ascending=True, axis=1)
        # break
        
        ## global signal regression:
        time_signal = time_signal.sub(time_signal.mean(axis = 1), axis = 0)
        
        ## hcp:
        # subject_id = re.search('TimeSeries(.*).csv', file_time_signal).group(1)
        # time_signal = pd.read_csv(file_time_signal, header = None, sep = ',')
        # time_signal = time_signal.T
        
        # print(subject_id)

        # check nan values:
        if time_signal.isna().any(axis = None):
            print('Nan value for time_signal:' + subject_id)
            spectrum_sub_list.append([])
            # test_break = True
            continue

        # we could use this to directly skip files with NaN values. not implemented yet...
        if fill_subject_id:
            subject_id_list.append(subject_id)
        
        ## compute inner product for each time point:
        P = np.matmul(time_signal.values, wavelets.values)
        spectrum_sub_list.append(P)
    
    # break
        
    fill_subject_id = False
    spectrum_list.append(spectrum_sub_list)

print('finished!')

ocd_out02_harmonics/CommonHarWavelets_Harmonics001.csv
ocd_out02_harmonics/CommonHarWavelets_Harmonics002.csv
ocd_out02_harmonics/CommonHarWavelets_Harmonics003.csv
ocd_out02_harmonics/CommonHarWavelets_Harmonics004.csv
ocd_out02_harmonics/CommonHarWavelets_Harmonics005.csv
ocd_out02_harmonics/CommonHarWavelets_Harmonics006.csv
ocd_out02_harmonics/CommonHarWavelets_Harmonics007.csv
ocd_out02_harmonics/CommonHarWavelets_Harmonics008.csv
ocd_out02_harmonics/CommonHarWavelets_Harmonics009.csv
ocd_out02_harmonics/CommonHarWavelets_Harmonics010.csv
ocd_out02_harmonics/CommonHarWavelets_Harmonics011.csv
ocd_out02_harmonics/CommonHarWavelets_Harmonics012.csv
ocd_out02_harmonics/CommonHarWavelets_Harmonics013.csv
ocd_out02_harmonics/CommonHarWavelets_Harmonics014.csv
ocd_out02_harmonics/CommonHarWavelets_Harmonics015.csv
ocd_out02_harmonics/CommonHarWavelets_Harmonics016.csv
ocd_out02_harmonics/CommonHarWavelets_Harmonics017.csv
ocd_out02_harmonics/CommonHarWavelets_Harmonics018.csv
ocd_out02_

In [38]:
time_signal

,0,1,2,3,4,5,6,7,8,9,...,106,107,108,109,110,111,112,113,114,115
0,0.225253,0.225169,1.612938,0.175553,0.638393,-1.433197,2.000672,0.073015,1.452435,-0.849939,...,-0.782703,-1.301354,4.281563,-0.713395,-0.539613,2.705890,0.211141,0.170254,-0.357286,-5.050809
1,0.092857,-0.601434,2.372192,-0.178943,0.590885,-2.803635,2.350774,-0.465846,1.761983,-1.126751,...,-0.845922,-0.213479,8.725738,-0.253465,-0.982970,3.034154,-0.755982,-0.997859,-3.566848,-9.810999
2,-0.134343,-1.440884,2.026170,-0.598084,-0.014288,-3.537894,0.869060,-1.810393,0.658749,-1.588203,...,0.459737,1.178236,5.378586,0.750955,-1.114396,1.882596,-2.062064,-2.054123,-6.442387,-13.436575
3,-0.219670,-1.548189,0.547574,-0.626616,-0.594220,-2.885841,-1.708453,-2.904800,-1.254084,-1.844146,...,2.625261,1.602766,-2.621702,1.952456,-0.165089,-0.109244,-2.801274,-2.312895,-6.338569,-14.001830
4,-0.028112,-0.891157,-1.197176,-0.097633,-0.478590,-0.944603,-3.687343,-2.672305,-2.599006,-1.364728,...,3.970663,1.085231,-8.458328,2.702696,1.866949,-1.965349,-2.353279,-1.649039,-2.733473,-10.288489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,0.057214,1.026681,-0.055013,0.526237,-0.984493,1.157398,-0.565137,0.078422,-1.522979,1.004797,...,1.682376,3.550035,-11.066636,-1.174829,0.116509,2.343770,0.724958,0.612990,1.426273,6.709911
226,0.748405,0.947072,-0.293572,0.013928,-1.582672,0.265069,-1.144173,-0.469949,-1.871510,0.881435,...,1.221844,2.819602,-10.133911,0.702448,0.603570,3.103189,0.168384,0.573481,1.746819,4.432417
227,1.050320,0.656444,-0.687703,-0.484987,-1.707757,-0.652704,-1.471064,-0.917019,-1.530510,0.110491,...,0.349469,1.178175,-7.211191,1.852718,0.589248,0.933846,-0.211648,0.486783,1.571972,2.078805
228,0.754881,0.199633,-0.913689,-0.824093,-1.314758,-1.197102,-1.538389,-1.233027,-0.739854,-0.691341,...,-0.135408,-0.180424,-4.740205,0.632650,0.213276,-1.619625,-0.104340,0.036979,1.183808,2.611964


In [39]:
## save spectrum for each subject and brain region:
        
for brain_index in range(len(spectrum_list)):
    if len(spectrum_list[brain_index])==0:
        continue
    for subj in range(len(spectrum_list[0])):
        P = spectrum_list[brain_index][subj]

        out_file_name = output_dir + '/spectrum_subj' + '{:03}'.format(subj) + '_roi_' + '{:03}'.format(brain_index) + '.csv'
        np.savetxt(out_file_name, np.array(P), delimiter = ',')


In [37]:
len(spectrum_list)

116

In [31]:
## make histogram

## calculate entropy